In [1]:
!nvidia-smi

In [2]:
import tensorflow
tensorflow.__version__

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.applications.densenet import DenseNet201
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
image_size = [224,224] 
train_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
test_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [5]:
denseNet = DenseNet201(include_top=False,input_shape=image_size+[3],)

In [6]:
for layers in denseNet.layers:
  layers.trainable=False

In [7]:
out_features = glob('../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/*')
features = len(out_features)
print( "The number of output classes are ",features)
print(os.listdir(train_path))

In [8]:
x = Flatten()(denseNet.output)
prediction = Dense(features,activation='softmax')(x)

In [9]:
model = Model(inputs=denseNet.input,outputs=prediction)
model.summary()

In [10]:
train_datagen= ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
)

test_datagen = ImageDataGenerator(
                        rescale=1./255)

In [11]:
train_data = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=image_size,
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
)

test_data = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=image_size,
    class_mode='categorical',
    batch_size=128,
    shuffle=True,
)

In [12]:
# tensorflow callbacks

callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0.0025, patience=2, verbose=1),
    ModelCheckpoint(filepath='crop_disease_detection_model.h5', monitor='val_loss', 
                    verbose=0, save_best_only=True,)
    
]


In [13]:
model.compile(optimizer=Adam(),loss=CategoricalCrossentropy(),metrics='accuracy')

In [14]:
model_history = model.fit(train_data,validation_data=test_data,epochs=3,callbacks=[callbacks])

In [15]:
# fit.predict(os.path.join('../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid/Apple___healthy/01b32971-5125-4982-98e2-22daa9ae864a___RS_HL 7977.JPG'))

In [16]:
model.evaluate(test_data)

In [17]:
# plotting the accuracy
import matplotlib.pyplot as plt
plt.plot(model_history.history['accuracy'], label='train acc')
plt.plot(model_history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [18]:
# plotting the loss

plt.plot(model_history.history['loss'], label='train loss')
plt.plot(model_history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [19]:
preds = model.predict(test_data)
preds

In [20]:
# predicting covid positive or negative by uploading image

from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
model_path = 'crop_disease_detection_model.h5'
image_path = '../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Apple_scab/00075aa8-d81a-4184-8541-b692b78d398a___FREC_Scab 3335.JPG'
img = image.load_img(image_path,target_size = [224,224])
x = image.img_to_array(img)
x=x/255
x = np.expand_dims(x, axis=0)
preds = model.predict(x)
preds

In [23]:
np.argmax(preds)

In [22]:
model.save('/kaggle/working/model_trained.h5')